<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/Random_forests_pretrained_CNN_Aus_habitat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random forest classifiers**

This script splits the data into the same train/val/test sets as used for the pretrained CNN and trained CNN. It then trains on the training data and generates the accuracy for the validation data 50 times. The best performing model of the 50 repeats is used to inference on the test data and this accuracy is reported. This proces is repeated 32 times to generate an accuracyfor all train/val/test split combinations. A confusion matrix of all these is then generated.

In [1]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# imports, probably some redundant packages
from __future__ import division

import sklearn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
from sklearn.model_selection import train_test_split
import statistics

from sklearn.metrics import confusion_matrix

import collections
from sklearn.metrics import ConfusionMatrixDisplay

import seaborn as sn
import matplotlib.pyplot as plt

In [9]:
# Upload the csv of features and copy the path
path = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/full_dataset_features/pretrained_CNN_aus.csv'

num_classes = 2
labels = ['High fish', 'Low fish']

In [10]:
data = pd.read_csv (path)
df = data.reset_index() 
df = df.iloc[: , 2:] #remove additional indata = pd.read_csv (path) #load dataframe
temp_df = data.reset_index() #put index in order
temp_df = temp_df.iloc[: , 2:] #remove unnecessary index
temp_df = temp_df.T #transpose to match indices format
temp_df = temp_df.reset_index() #re-add the index
df = temp_df.rename(columns={"index": "minute"})

df

,minute,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,SiteJdeployment2.1677983769.181103032034.wav,0.000929,0.093042,0.285380,0.210208,0.000000,0.0,0.0,0.006659,0.000000,...,0.0,0.023046,0.000220,0.000000,0.000000,0.225300,0.304314,0.000000,0.000000,0.0
1,SiteAdeployment1.805322778.181023172733.wav,0.030780,0.100007,0.408427,0.050889,0.000000,0.0,0.0,0.002414,0.000000,...,0.0,0.060092,0.010739,0.000000,0.028925,0.244283,0.167082,0.000000,0.000000,0.0
2,SiteIdeployment2.805322778.181102073302.wav,0.023870,0.043412,0.331813,0.121506,0.000000,0.0,0.0,0.000000,0.000255,...,0.0,0.043277,0.000077,0.000000,0.016917,0.357364,0.186497,0.000000,0.000000,0.0
3,SiteDdeployment4.805322778.181123173922.wav,0.039953,0.076123,0.304760,0.088957,0.001289,0.0,0.0,0.014608,0.000000,...,0.0,0.078101,0.053534,0.001675,0.013040,0.177259,0.129732,0.000000,0.000000,0.0
4,SiteEdeployment6.1677983769.181209085140.wav,0.005541,0.105616,0.485045,0.074837,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.056985,0.008627,0.000000,0.020391,0.282245,0.129462,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8970,SiteEdeployment4.1677983769.181123193512.wav,0.005815,0.111684,0.373035,0.186946,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.085167,0.000000,0.000137,0.029835,0.296419,0.188369,0.000000,0.001338,0.0
8971,SiteBdeployment2.1678278701.181031081236.wav,0.001825,0.060861,0.343262,0.226211,0.000142,0.0,0.0,0.000000,0.000000,...,0.0,0.030873,0.000000,0.000467,0.000219,0.443077,0.304441,0.000000,0.000000,0.0
8972,SiteFdeployment1.805322778.181024160925.wav,0.036383,0.040715,0.350639,0.109169,0.001251,0.0,0.0,0.021016,0.000000,...,0.0,0.080465,0.019677,0.000822,0.004993,0.170270,0.152097,0.000000,0.000000,0.0
8973,SiteHdeployment2.805322778.181103064253.wav,0.024534,0.068714,0.446816,0.091898,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.165965,0.007297,0.000000,0.196851,0.172095,0.139393,0.000565,0.000000,0.0


In [11]:
def get_class(filename):
    #find part of the name that corresponds to the deployment
     #adapted the get_identifier function above to only get class (e.g healthy)
    site = filename[0:5]
    return site
    
df_withclasses = df

# add a column to the DF that contains class
new_list = []
for i in range(len(df_withclasses)):
    new_list.append(get_class(df_withclasses['minute'][i]))

df_withclasses.insert(1, 'Class', new_list)
#df_withclasses.head(20)

pretrained_df = df_withclasses 

# Remove the 4 sites not being used for fish habitat comparison
pretrained_df.drop(pretrained_df.index[pretrained_df['Class'] == 'SiteC'], inplace=True)
pretrained_df.drop(pretrained_df.index[pretrained_df['Class'] == 'SiteD'], inplace=True)
pretrained_df.drop(pretrained_df.index[pretrained_df['Class'] == 'SiteE'], inplace=True)
pretrained_df.drop(pretrained_df.index[pretrained_df['Class'] == 'SiteI'], inplace=True)

# Rename the renaming sites to High or Low fish
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteA'],'Low fish')
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteB'],'Low fish')
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteG'],'Low fish')
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteJ'],'Low fish')

pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteF'],'High fish')
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteH'],'High fish')
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteK'],'High fish')
pretrained_df['Class'] = pretrained_df['Class'].replace(['SiteL'],'High fish')

pretrained_df.reset_index(drop=True, inplace=True)
pretrained_df.head()

,minute,Class,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,SiteJdeployment2.1677983769.181103032034.wav,Low fish,0.000929,0.093042,0.285380,0.210208,0.0,0.0,0.0,0.006659,...,0.0,0.023046,0.000220,0.000000,0.000000,0.225300,0.304314,0.0,0.0,0.0
1,SiteAdeployment1.805322778.181023172733.wav,Low fish,0.030780,0.100007,0.408427,0.050889,0.0,0.0,0.0,0.002414,...,0.0,0.060092,0.010739,0.000000,0.028925,0.244283,0.167082,0.0,0.0,0.0
2,SiteBdeployment5.1677983769.181201071827.wav,Low fish,0.002083,0.104573,0.366874,0.168646,0.0,0.0,0.0,0.001900,...,0.0,0.039408,0.008483,0.002023,0.026743,0.295693,0.211083,0.0,0.0,0.0
3,SiteAdeployment6.805322778.181208200239.wav,Low fish,0.028723,0.063788,0.302572,0.160899,0.0,0.0,0.0,0.003868,...,0.0,0.077238,0.007906,0.000191,0.039737,0.211901,0.197882,0.0,0.0,0.0
4,SiteJdeployment3.1677983769.181110075705.wav,Low fish,0.001223,0.154777,0.341569,0.162932,0.0,0.0,0.0,0.000000,...,0.0,0.026811,0.000020,0.000000,0.003966,0.317349,0.288543,0.0,0.0,0.0


In [12]:
def similar(x,y):
    si = 0
    for a,b in zip(x, y):
        if a == b:
            si += 1
    return (si/len(x)) * 100

def get_class(filename):
    #find part of the name that corresponds to the deployment
     #adapted the get_identifier function above to only get class (e.g healthy)
    site = filename[0:5]
    return site
    
df_withclasses = df

# add a column to the DF that contains class
new_list = []
for i in range(len(df_withclasses)):
    new_list.append(get_class(pretrained_df['minute'][i]))

pretrained_df.insert(1, 'Site', new_list)

In [13]:
all_saved_test_accs = []
whole_site_saved_test_accs = []
ConfusionMatrix = np.zeros((num_classes, num_classes), dtype=float)

for i in range(4):  #4 sites in the lowfish class
  low_fish = ['SiteA', 'SiteB', 'SiteG', 'SiteJ'] #the 4 low fish sites
  remove_low = low_fish[i] #select 1 to be the val site
  train_low_fish = low_fish 
  train_low_fish.remove(remove_low) #remove the val site to get the train lowfish sites
  for k in range(4): #4 sites in the highfish class
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []

    high_fish = ['SiteH', 'SiteF', 'SiteK', 'SiteL'] #the 4 high fish sites
    remove_high = high_fish[k] #select 1 to be the test site
    train_high_fish = high_fish 
    train_high_fish.remove(remove_high) #remove the test site to get the train lowfish sites
    
    # Make the train df's, starting with low fish then append high fish
    train_df = indices_df.loc[indices_df['Site'].isin(train_low_fish)]
    train_df = train_df.append(indices_df.loc[indices_df['Site'].isin(train_high_fish)])

    # create val df that is 50:50 low and high fish
    lowfish_df = indices_df.loc[indices_df['Site'] == remove_low]
    highfish_df = indices_df.loc[indices_df['Site'] == remove_high]
    val_df = lowfish_df.iloc[::2,:] # take every other lowfish minute and create a new df
    val_df = val_df.append(highfish_df.iloc[::2,:])
    # create test df's that is the ramaining 50:50 low and high fish
    test_df = lowfish_df.iloc[1::2,:] # take every other lowfish minute and create a new df
    test_df = test_df.append(highfish_df.iloc[1::2,:])

    #shuffle the df's so the RF's aren't first fed all low then fed all high fish after
    np.random.seed(123) 
    train_df = train_df.sample(frac=1)
    val_df = val_df.sample(frac=1)
    test_df = test_df.sample(frac=1)
    np.random.seed() #now lift the seed so that randomisation can be used again in the rest of the script

    # Make new df's of the feats and labels to feed random forests
    train_feats = train_df.iloc[:, 3:].to_numpy()
    train_labels = train_df.iloc[:, 2].to_numpy()

    val_feats = val_df.iloc[:, 3:].to_numpy()
    val_labels = val_df.iloc[:, 2].to_numpy()

    test_feats = test_df.iloc[:, 3:].to_numpy()
    test_labels = test_df.iloc[:, 2].to_numpy()

    
    val_accuracy_score = 0
    for k in range(50):              # Picked 50 as 50 epochs used in NN
      #Training on train data
      model = RandomForestClassifier(n_jobs = -1, random_state=k)
      model.fit(train_feats, train_labels)

      #Inferencing on validation data
      new_val_acc = model.score(val_feats, val_labels)

      #If val acc improved, inference on test data
      if new_val_acc >  val_accuracy_score:
        #save the new high score for val
        val_accuracy_score = new_val_acc 

        #get 1min acc for test data
        test_acc = model.score(test_feats, test_labels) 
        
        #get the acc when using the most common prediction across all minutes in a day for each site
        test_predictions = model.predict(test_feats)

        # get confusion matrix values
        best_ConfusionMatrix = confusion_matrix(test_labels, test_predictions, labels = labels) ###

        if  similar(test_labels, test_predictions) >= 50:
          whole_site_acc = 1
        else:
          whole_site_acc = 0
        
      
    #Save best results
    ConfusionMatrix = np.add(ConfusionMatrix, best_ConfusionMatrix) ###
    thisCV_saved_test_accs.append(test_acc)
    all_saved_test_accs.append(test_acc)
    thisCV_saved_test_day_accs.append(whole_site_acc)
    whole_site_saved_test_accs.append(whole_site_acc)

    print('Accuracies for 1min files from this site:')
    print(thisCV_saved_test_accs)
    print('Accuracies for the whole site [0 or 1]:')
    print(thisCV_saved_test_day_accs)
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []
    print()





pt1_saved_test_accs = all_saved_test_accs
pt1_whole_site_saved_test_accs = whole_site_saved_test_accs


# this will run all the lowfish sites as val, then repeat the whole thing below but swap the highfish to be in the top loop and low fish in bottom loop
    
    
print('Part 1 Completed RFs:')
print(len(all_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(all_saved_test_accs)
stdev = np.std(all_saved_test_accs)

print('1min saved_test_accs: ')
print(all_saved_test_accs)
result = 'Mean accuracy for 1min files with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result)     



print('Part 1 Completed RFs:')
print(len(whole_site_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(whole_site_saved_test_accs)
stdev = np.std(whole_site_saved_test_accs)

print('Whole site saved test_accs: ')
print(whole_site_saved_test_accs)
result = 'Mean accuracy for whole sites with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result)


all_saved =  mean_accuracy
stdev_pt1 = stdev






all_saved_test_accs = []
whole_site_saved_test_accs = []

for i in range(4):  #4 sites in the lowfish class
  high_fish =  ['SiteH', 'SiteF', 'SiteK', 'SiteL'] #the 4 high fish sites
  remove_high = high_fish[i] #select 1 to be the val site
  train_high_fish = high_fish 
  train_high_fish.remove(remove_high) #remove the val site to get the train lowfish sites
  #print(train_low_fish)
  for k in range(4): #4 sites in the highfish class
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []

    low_fish = ['SiteA', 'SiteB', 'SiteG', 'SiteJ'] #the 4 low fish sites
    remove_low = low_fish[k] #select 1 to be the test site
    train_low_fish = low_fish 
    train_low_fish.remove(remove_low) #remove the test site to get the train lowfish sites
    #print(train_high_fish)
    
    # Make the train df's, starting with low fish then append high fish
    train_df = indices_df.loc[indices_df['Site'].isin(train_low_fish)]
    train_df = train_df.append(indices_df.loc[indices_df['Site'].isin(train_high_fish)])

    # create val df that is 50:50 low and high fish
    lowfish_df = indices_df.loc[indices_df['Site'] == remove_low]
    highfish_df = indices_df.loc[indices_df['Site'] == remove_high]
    val_df = lowfish_df.iloc[1::2,:] # take every other lowfish minute and create a new df
    val_df = val_df.append(highfish_df.iloc[1::2,:])
    # create test df's that is the ramaining 50:50 low and high fish
    test_df = lowfish_df.iloc[::2,:] # take every other lowfish minute and create a new df
    test_df = test_df.append(highfish_df.iloc[::2,:])

    #shuffle the df's so the RF's aren't first fed all low then fed all high fish after
    np.random.seed(123) 
    train_df = train_df.sample(frac=1)
    val_df = val_df.sample(frac=1)
    test_df = test_df.sample(frac=1)
    np.random.seed() #now lift the seed so that randomisation can be used again in the rest of the script

    # Make new df's of the feats and labels to feed random forests
    train_feats = train_df.iloc[:, 3:].to_numpy()
    train_labels = train_df.iloc[:, 2].to_numpy()

    val_feats = val_df.iloc[:, 3:].to_numpy()
    val_labels = val_df.iloc[:, 2].to_numpy()

    test_feats = test_df.iloc[:, 3:].to_numpy()
    test_labels = test_df.iloc[:, 2].to_numpy()

    
    val_accuracy_score = 0
    for k in range(50):              # Picked 50 as 50 epochs used in NN
      #Training on train data
      model = RandomForestClassifier(n_jobs = -1, random_state=k)
      model.fit(train_feats, train_labels)

      #Inferencing on validation data
      new_val_acc = model.score(val_feats, val_labels)

      #If val acc improved, inference on test data
      if new_val_acc >  val_accuracy_score:
        #save the new high score for val
        val_accuracy_score = new_val_acc 

        #get 1min acc for test data
        test_acc = model.score(test_feats, test_labels) 
        
        #get the acc when using the most common prediction across all minutes in a day for each site
        test_predictions = model.predict(test_feats)

        # get confusion matrix values
        best_ConfusionMatrix = confusion_matrix(test_labels, test_predictions, labels = labels) ###

        if  similar(test_labels, test_predictions) >= 50:
          whole_site_acc = 1
        else:
          whole_site_acc = 0
        
      
    #Save best results
    ConfusionMatrix = np.add(ConfusionMatrix, best_ConfusionMatrix) ###
    thisCV_saved_test_accs.append(test_acc)
    all_saved_test_accs.append(test_acc)
    thisCV_saved_test_day_accs.append(whole_site_acc)
    whole_site_saved_test_accs.append(whole_site_acc)

    print('Accuracies for 1min files from this site:')
    print(thisCV_saved_test_accs)
    print('Accuracies for the whole site [0 or 1]:')
    print(thisCV_saved_test_day_accs)
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []
    print()






both_saved_test_accs = pt1_saved_test_accs + all_saved_test_accs 
both_whole_site_saved_test_accs = pt1_whole_site_saved_test_accs + whole_site_saved_test_accs
    
print('Part 2 Completed RFs:')
print(len(both_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(both_saved_test_accs)
stdev = np.std(both_saved_test_accs)

print('1min saved_test_accs: ')
print(both_saved_test_accs)
result = 'Mean accuracy for 1min files with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result) 




print('Part 2 Completed RFs:')
print(len(both_whole_site_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(both_whole_site_saved_test_accs)
stdev = np.std(both_whole_site_saved_test_accs)

print('Whole site saved test_accs: ')
print(both_whole_site_saved_test_accs)
result = 'Mean accuracy for whole sites with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result) 

print()
print(repr(ConfusionMatrix))

Accuracies for 1min files from this site:
[0.6675461741424802]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.596537949400799]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.6186666666666667]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.6728232189973615]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.7814285714285715]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.6738816738816739]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.791907514450867]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.7185714285714285]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from this site:
[0.7562189054726368]
Accuracies for the whole site [0 or 1]:
[1]

Accuracies for 1min files from

In [ ]:
#HPCconfusionmatrix = [[9138., 3742.],
      # [3100., 8444.]])

array = ConfusionMatrix

df_cm = pd.DataFrame(array, index = ["Fish rich", "Fish poor"],
                  columns = ["Fish rich", "Fish poor"])
plt.figure(figsize = (15,12))
cmap = sn.cm.rocket_r
ax = sn.heatmap(df_cm, annot=True, annot_kws={"fontsize":25}, fmt='g', cmap = cmap)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 25)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 25)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)